# PyMixer Tutorial

This notebook contains code snippets demonstrating how `pymixer` package can be used.

### Step 0. Setup

Let us import required modules, classes, and functions.

In [ ]:
import itertools
import os

import IPython.display
import matplotlib.pyplot as plt
import numpy as np
import pretty_midi
from pymixer.project import MidiTrackSpec, Project
from pymixer.sound_makers import FluidsynthSoundMaker
from sinethesizer.synth.core import Event
from sinethesizer.effects.reverb import apply_room_reverb
from sinethesizer.effects.stereo import apply_stereo_to_mono_conversion
from sinethesizer.io.events_to_wav import write_timeline_to_wav

In [ ]:
%matplotlib inline

Then, put all input data to a single directory. In this tutorial, input data are three MIDI files constituting an introductory section of a fugue. This files are included in the repository as binary assets, so the below cell is valid even if it is left unchanged.

In [ ]:
input_dir = os.path.join(os.getcwd(), 'assets', 'tutorial_inputs')
intermediate_dir = os.path.join(os.getcwd(), 'assets')
output_dir = os.path.join(os.getcwd(), 'assets')

As it is known, MIDI files do not contain any actual sounds; they contain only instructions how to produce it. Here, FluidSynth is used to generate audio output from MIDI files. Since FluidSynth is a soundfont player, a soundfont file is required. Pipe organ soundfonts look appropriate for our fugue sample. A soundfont based on the organ located at Pitea School of Music [is publicly available](https://stratmaninstruments.wordpress.com/swedish-organ-series) under Creative Commons Attribution-ShareAlike 2.5 license. Let us use it (update the path below if you saved the soundfont file to other place). 

In [ ]:
soundfont_path = os.path.join(
    os.path.expanduser('~'), 'sound', 'soundfonts', 'j3.20_PiteaMHS_3.0', 'Pitea_3.0.sf2'
)

### Step 1. Configuration

Within `pymixer` domain, a track is a 2-channel audio with independent gain control and independent effects. In the next cell, it is defined how to make tracks from original MIDI files. Each track corresponds to its specification which is an instance of `MidiTrackSpec` class.

This instance is responsible for two aspects:
* how to produce sound from MIDI;
* which input files to use.

The latter thing is simple: a track uses contiguous sequence of input files and so it is enough to set the first one and the last one inclusively. In the below example, it is enough to put everywhere '01.mid' as a start and '03.mid' as an end. However, some of the ranges can be shorten, because fugue voices introduce one after the other.

As for sound production, instance of either `FluidsynthSoundMaker` class or `SinethesizerSoundMaker` class is needed. Let us look closer at the former one, since only FluidSynth is used in this demo. The following things are configured below:
* path to save an intermediate MIDI file that stores only relevant to the current track data (i.e., no other instruments and no preceeding or following events);
* path to the soundfont;
* dictionary that says which instruments to keep and which programs to use for playing them; for example, the first track includes only the instrument named '1' in the MIDI files and this instrument is mapped to program 15 ("Gedackt 8'" in the Pitea soundfont), whereas the second track includes exactly the same instrument, but now played with program 18 ("Principal 4'"); having two tracks with the same notes but different programs allows independent processing of this programs;
* flags whether to use some built-in FluidSynth effects.

In [ ]:
tracks_specs = [
    MidiTrackSpec(
        FluidsynthSoundMaker(
            f"{intermediate_dir}/track_1.mid",
            soundfont_path,
            {'1': 15},
            fluidsynth_chorus=True
        ),
        '02.mid',
        '03.mid'
    ),
    MidiTrackSpec(
        FluidsynthSoundMaker(
            f"{intermediate_dir}/track_2.mid",
            soundfont_path,
            {'1': 18},
            fluidsynth_chorus=True
        ),
        '02.mid',
        '03.mid'
    ),
    MidiTrackSpec(
        FluidsynthSoundMaker(
            f"{intermediate_dir}/track_3.mid",
            soundfont_path,
            {'2': 5},
            fluidsynth_chorus=True
        ),
        '01.mid',
        '03.mid'
    ),
    MidiTrackSpec(
        FluidsynthSoundMaker(
            f"{intermediate_dir}/track_4.mid",
            soundfont_path,
            {'2': 6},
            fluidsynth_chorus=True
        ),
        '01.mid',
        '03.mid'
    ),
    MidiTrackSpec(
        FluidsynthSoundMaker(
            f"{intermediate_dir}/track_5.mid",
            soundfont_path,
            {'3': 23},
            fluidsynth_chorus=True
        ),
        '03.mid',
        '03.mid'
    ),
    MidiTrackSpec(
        FluidsynthSoundMaker(
            f"{intermediate_dir}/track_6.mid",
            soundfont_path,
            {'3': 25},
            fluidsynth_chorus=True
        ),
        '03.mid',
        '03.mid'
    ),
]

Then tracks configuration must be passed to a mixing project represented by `Project` class.

Optional argument `offsets` sets interlocation of each pair of adjacent MIDI inputs. Positive values stand for pauses (caesuras, in musical jargon) and negative values stand for overlappings: the *(i+1)*-th file starts before the *i*-th file ends.

Please note that all opening and trailing silences from MIDI files are ignored by `pymixer`. Use `offsets` argument to separate parts with pauses. As for opening and trailing silences for the output as a whole, use eponymous arguments of `mix` method (see further).

In [ ]:
project = Project(
    input_dir,
    tracks_specs,
    offsets=[0.0, 0.0]
)

At this time, all MIDI inputs are converted to audio stored in-memory as `numpy` arrays. Now, the mixing itself starts.

### Step 2. Applying effects to tracks

Actually, every function that takes `numpy` array as input and returns `numpy` array as output can be used for applying effects. In this tutorial, `sinethesizer` effects are chosen as an example. It is supposed that our goal is to erase original panning (if any) and then introduce a new panning with a room reverb so that all tracks based on the instrument named '1' are located to the left, all tracks based on the instrument named '2' are located to the right and all tracks based on the instrument named '3' are centered.

A `sinethesizer` event is needed for all `sinethesizer` effects. For some effects, this event affects the output, but this is not the case for reverb and stereo-to-mono conversion. So let us create a placeholder. 

In [ ]:
dummy_event = Event(
    instrument='does_not_matter',
    start_time=0,
    duration=1,
    frequency=1,
    velocity=1,
    effects="",
    frame_rate=project.frame_rate
)

The next cell configures three reverb presets with reflections of sound waves within a room being simulated.

In [ ]:
reverb_params_by_name = {
    'left': {
        "room_length": 65, "room_width": 25, "room_height": 15,
        "reflection_decay_factor": 0.7, "sound_speed": 343,
        "listener_x": 45, "listener_y": 12.5, "listener_z": 1.7,
        "listener_direction_x": 1, "listener_direction_y": 0,
        "sound_source_x": 57.124355652982146, "sound_source_y": 19.5, "sound_source_z": 4,
        "sound_source_direction_x": -0.8660254037844387, "sound_source_direction_y": -0.49999999999999994, "sound_source_direction_z": 0,
        "angle": 1.5707963267948966, "n_reflections": 30
    },
    'center': {
        "room_length": 65, "room_width": 25, "room_height": 15,
        "reflection_decay_factor": 0.7, "sound_speed": 343,
        "listener_x": 45, "listener_y": 12.5, "listener_z": 1.7,
        "listener_direction_x": 1, "listener_direction_y": 0,
        "sound_source_x": 59.0, "sound_source_y": 12.5, "sound_source_z": 4,
        "sound_source_direction_x": -1.0, "sound_source_direction_y": -0.0, "sound_source_direction_z": 0,
        "angle": 1.5707963267948966, "n_reflections": 30
    },
    'right': {
        "room_length": 65, "room_width": 25, "room_height": 15,
        "reflection_decay_factor": 0.7, "sound_speed": 343,
        "listener_x": 45, "listener_y": 12.5, "listener_z": 1.7,
        "listener_direction_x": 1, "listener_direction_y": 0,
        "sound_source_x": 57.124355652982146, "sound_source_y": 5.500000000000001, "sound_source_z": 4,
        "sound_source_direction_x": -0.8660254037844387, "sound_source_direction_y": 0.49999999999999994, "sound_source_direction_z": 0,
        "angle": 1.5707963267948966, "n_reflections": 30
    },
}
instrument_name_to_reverb_params = {
    '1': reverb_params_by_name['left'],
    '2': reverb_params_by_name['right'],
    '3': reverb_params_by_name['center'],
}

Now, apply the effects. Initial panning is erased with stereo-to-mono conversion and then room reverb creates new panning.

In [ ]:
instrument_names = [
    k
    for track_spec in tracks_specs
    for k in track_spec.sound_maker.instrument_name_to_program.keys()
]
for index, instrument_name in enumerate(instrument_names):
    project.tracks[index].timeline = apply_stereo_to_mono_conversion(
        project.tracks[index].timeline,
        dummy_event
    )
    project.tracks[index].timeline = apply_room_reverb(
        project.tracks[index].timeline,
        dummy_event,
        **instrument_name_to_reverb_params[instrument_name]
    )

### Step 3. Evaluation of the mix

Below, track gains can be changed easily until mixed sound is good enough. Just execute the cell iteratively as many times as needed.

In [ ]:
gains = [1.0, 1.0, 1.0, 1.0, 1.2, 1.2]
opening_silence = 0.5
trailing_silence = 0.5

timeline = project.mix(gains, opening_silence, trailing_silence)
timeline /= np.max(np.abs(timeline))
IPython.display.Audio(timeline, rate=project.frame_rate)

As proverb says, it is better to see something once than to hear about it multiple times. Let us inspect the mix visually as well.

In [ ]:
grouped_track_indices = []
for _, group in itertools.groupby(sorted(enumerate(instrument_names), key=lambda x: x[1]), key=lambda x: x[1]):
    grouped_track_indices.append([x[0] for x in group])
timelines = project.mix(gains, opening_silence, trailing_silence, grouped_track_indices)

unique_instrument_names = set(instrument_names)
fig = plt.figure(figsize=(15, 3 * len(unique_instrument_names)))
first_ax = None
for i, instrument_name in enumerate(sorted(unique_instrument_names)):
    ax_location = len(unique_instrument_names) * 100 + 11 + i
    if first_ax is None:
        ax = fig.add_subplot(ax_location)
        first_ax = ax
    else:
        ax = fig.add_subplot(ax_location, sharey=first_ax)
    mono_timeline = np.sum(timelines[(2 * i):(2 * i + 1), :], axis=0) / 2  # Average over channels is visualized.
    ax.plot(mono_timeline)
    for track in project.tracks:
        ax.axvline(project.frame_rate * (track.start_time + opening_silence), c='red')

### Step 4. Saving WAV output

In [ ]:
write_timeline_to_wav(f'{output_dir}/result.wav', timeline, project.frame_rate)